In [1]:
import mysql.connector as mysql
import pandas as pd

db = mysql.connect(
    host = "localhost",
    port = '3306',
    user = "local",
    password = "local",
    database = "mail_data"
)

In [ ]:
cursor = db.cursor()
command = f"SELECT * FROM mails"
cursor.execute(command)
df = pd.DataFrame(cursor.fetchall(), columns = ['id', 'message_id', 'label_ids', 'date', 'sender', 'subject', 'body', 'category', 'action'])

# save the dataframe to a csv file
df.to_csv('../mount/mails.csv')

cursor.close()

In [ ]:
df = pd.read_csv('../mount/mails.csv')
print(df.shape)
df.drop_duplicates(subset = 'message_id', keep = 'first', inplace = True)
print(df.shape)

In [ ]:
print(df['subject'].isnull().sum())
print(df['category'].isnull().sum())
print(df['action'].isnull().sum())

In [ ]:
df = df.dropna(subset=['subject'])
print(df.shape)

In [ ]:
df.columns

In [12]:
df = df[['message_id', 'subject', 'category', 'action']]

In [ ]:
df

In [38]:
from collections import Counter as counter
words = counter(df['subject'].str.cat(sep = ' ').split())

In [ ]:
words.most_common(40)

### Note:
As the corpus is a small one and consists of personal mails from recurring senders with similar topics, applying a simple word frequency representation of the words would severly overfit the model

# Preprocessing Text

In [ ]:
import re 

def preprocess(text):
    assert type(text) == str, f"input {text} is not a string and has type {type(text)}"
    text = text.lower() # lowercase
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = re.sub(r'\d+', '', text) # remove numbers
    text = text.strip() # remove whitespaces
    return text

df['subject'] = df['subject'].apply(preprocess)
df

In [ ]:
df.groupby(by='action')['message_id'].count()

In [ ]:
df.groupby(by='category').count()

### Note:
as many categories have very small frequencies, for a better training and results, certain categories have to be clubbed together.
<br><br>
In Actions classes would be: {'READ': ['READ'], 'IGNORE': ['IGNORE'], 'ACT': ['URGENT'. 'RESPOND', 'ACT', 'IMPORTANT']}
<br><br>
Ic category, the classes would be: {'Education': ['Education'], 'Newsletters': ['Newsletters', 'News'], 'Personal': ['Personal'], 'Promotions': ['Promotions'], 'Social': ['Social', 'Forums'], 'Work':['Work'], 'Unknown': ['Unknown', 'Security']}

In [54]:
action_classes = {'READ': 'READ',
                  'IGNORE': 'IGNORE',
                  'URGENT': 'ACT',
                  'ACT': 'ACT',
                  'RESPOND': 'ACT',
                  'IMPORTANT': 'ACT'}
category_classes = {'Education': 'Education',
                    'Newsletters': 'Newsletters',
                    'News': 'Newsletters',
                    'Personal': 'Personal',
                    'Promotions': 'Promotions',
                    'Social': 'Social',
                    'Forums': 'Social',
                    'Work': 'Work',
                    'Unknown': 'Unknown',
                    'Security': 'Unknown'}

def apply_action_class(action):
    return action_classes[action]
def apply_category_class(category):
    return category_classes[category]

final_df = df[:]
final_df['action'] = final_df['action'].apply(apply_action_class)
final_df['category'] = final_df['category'].apply(apply_category_class)

In [ ]:
final_df

In [60]:
final_df.to_csv('../mount/mails_processed.csv')

## Train-test split: 
Create two separate datasets 95 / 5 split for train / test 

In [4]:
from sklearn.model_selection import train_test_split 
import pandas as pd

In [46]:
df = pd.read_csv('../mount/mails_processed.csv')

train_df, test_action_df = train_test_split(df, test_size=0.025, stratify=df['action'])
train_df, test_category_df = train_test_split(df, test_size=0.025, stratify=df['category'])

test_df = pd.concat((test_action_df, test_category_df), ignore_index=True)

In [ ]:
train_df.groupby(by='category').count()

In [ ]:
test_df.groupby(by='category').count()

In [52]:
train_df.to_csv(f'../mount/train_data.csv')
test_df.to_csv(f'../mount/test_data.csv')